STABLE = all vars stable(may still have some red Any in output)

Graph Generation Functions

In [ ]:
using LightGraphs

"""
STABLE
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
STABLE
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
STABLE
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L::SparseMatrixCSC{Int64,Int64} = laplacian_matrix(g)
    a::Array{Float64,1} = lufact(L) \ (b - mean(b))
    p::Array{Float64,1} = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
STABLE
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end
;

Optimization Functions

In [ ]:
"""
STABLE
soft threshold
"""
soft(c,lambda) = sign(c).*max(abs(c)-lambda/2,0)

"""
STABLE
computes gradient
"""
function gradient2(a,a_0,u,L,rho,b,y)
    convb::Array{Float64,2} =zeros(length(b),1)+b
    grad::Array{Float64,2} = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-convb)
    return grad
end;


function gradient(a,a_0,u,L,rho,b,y)
    grad = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-b)
    return grad
end;



"""
STABLE
computes hessian
"""
function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return hess
end;




"""
STABLE
newton raphson for a update
"""
function newton(y_i,a_0,L,rho,b,u)
    a::Array{Float64,2} = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - inv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    if(iters == MAX_ITER)
        print("max iter reached")
    end
    return a
end
;




ADMM Function

In [ ]:
"""
2 Temp Vars UNSTABLE
ADMM Function 
Parameters:
A - Array of Array{Int64,2} of connections where each column is connections at time t
L - Array of SparseMatrixCSC(Int64,Int64) where each index i is the laplacian at time t= i-1
t - number of ending nodes
t_0 - number of starting nodes
lambda,rho -  Float64s for tuning and optimization
"""
function ADMM(A,L,t,t_0, rho, lambda)
    new = t-t_0
    a = zeros(t-1,new)
    b::Array{Float64,2} = zeros(t_0,1)
    u = zeros(t-1,new)
    iters = 0
    diff = 1.0
    b_old = b;
	while(diff >STOP_DIFF && iters< MAX_ITER )
        #a update
        for i in 1:new    
            a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:(t_0 +i-1),i])
        end
        #b update
        c = zeros(t-1,1)
        for i in 1:new
            c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ (u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i]))/(rho*new)
        end
        b = soft(c[1:t_0],2*lambda/rho)
        #u update
        for i in 1:new
            u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-[b' zeros(i-1)']')
        end
        diff  = norm(b-b_old)
        b_old = b
        println(diff)
	end
	return b
end
;

Example Graph Generation and Optimization Parameters for 5 nodes

In [ ]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*5. 
b[1] = 5
genb = copy(b)  # save for later
g = randEdgeGen(g,10000)
A = Array{Int64,2}[]
L =  SparseMatrixCSC{Int64,Int64}[]
push!(L, laplacian_matrix(g))
numnewnodes = 1
a_0 = -4
# creates matrix A and L where A[i] is the connections for ith node and L[i] is the laplacian of the i-1st time step 
for i in 1:numnewnodes  
    g = addPrefNode(g,b, a_0)
    push!(L,laplacian_matrix(g))
    connects = zeros(2^levels-2+i,1)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end


t = 2^levels-1+numnewnodes #number of nodes at time t
t_0 = 2^levels-1  # number of initial nodes
;

const MAX_ITER = 1000
const STOP_DIFF = 0.001;

rho = 1
lambda = 0.0005
new = numnewnodes



Example ADMM

In [ ]:
ADMM(A,L,t,t_0,rho,lambda)